In [ ]:
! pip install datasets
! pip install -U 'accelerate==0.27.0'
! pip install -U 'transformers==4.41.2'

In [ ]:
from datasets import load_dataset
from transformers import BertTokenizer, EncoderDecoderModel, Trainer, TrainingArguments
import torch
from transformers import get_linear_schedule_with_warmup
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from accelerate import Accelerator
from torch.utils.data import DataLoader

# Inicializar Accelerator
accelerator = Accelerator(mixed_precision='fp16')


# Carregar o dataset
all_dataset = load_dataset("VanessaSchenkel/translation-en-pt", field="data")

dataset = all_dataset['train'].shuffle(seed=42).select(range(int(len(all_dataset['train']) * 0.50)))

# Função para preparar os dados no formato correto
def preprocess_function(examples):
    # Extrair inputs e targets
    inputs = [ex["portuguese"] for ex in examples["translation"]]
    targets = [ex["english"] for ex in examples["translation"]]

    # Tokenizar inputs
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True)

    # Tokenizar targets e process labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, padding="max_length", truncation=True)["input_ids"]

    # Ensure labels match the input length and shape
    model_inputs["labels"] = labels

    return model_inputs

# Carregar o tokenizer e o modelo BERT
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = EncoderDecoderModel.from_encoder_decoder_pretrained(model_name, model_name)

# Definir o decoder_start_token_id
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias

In [ ]:
# Tokenizar o dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

In [ ]:
# Criação de DataLoader para treinamento
from torch.utils.data import DataLoader
train_dataloader = DataLoader(tokenized_datasets, batch_size=16, shuffle=True)

In [ ]:

# Configurar os parâmetros de treinamento com ajuste adicional
training_args = TrainingArguments(
    output_dir="./drive/MyDrive/FIAP/LLMs/Aula 2/results",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=1000,
    fp16=True,  # Habilitar mixed precision training
    gradient_accumulation_steps=2,
)


# Inicializar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets
)



In [ ]:
# Treinar o modelo
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:623: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step,Training Loss
1000,0.328000


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:623: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.10/dist-packages/tra

Step,Training Loss
1000,0.328000
2000,0.044800
3000,0.035800


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:623: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.10/dist-packages/tra

TrainOutput(global_step=3256, training_loss=0.1281803055535956, metrics={'train_runtime': 3335.2829, 'train_samples_per_second': 15.62, 'train_steps_per_second': 0.976, 'total_flos': 3.197275228274688e+16, 'train_loss': 0.1281803055535956, 'epoch': 1.0})

In [ ]:

# Salvar o modelo
trainer.save_model("./drive/MyDrive/FIAP/LLMs/Aula 2/bert-translation-en-pt-v3")
tokenizer.save_pretrained("./drive/MyDrive/FIAP/LLMs/Aula 2/bert-tokenizer-translation-en-pt-v3")